In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None, 'display.max_columns', None)

## Reading in the original Netflix data set
movie = pd.read_csv('Movie_Data.csv')

## Printing the first five observations
movie.head()

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,Actors,View Rating,IMDb Score,Rotten Tomatoes Score,Metacritic Score,Awards Received,Awards Nominated For,Boxoffice,Release Date,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,"Lina Leandersson, Kåre Hedebrant, Per Ragnar, ...",R,7.9,98.0,82.0,74.0,57.0,"$2,122,065",12 Dec 2008,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,https://www.youtube.com/watch?v=LqB6XJix-dM,YouTube
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,"Cleo, Paddy Considine, Beanie Feldstein, Dónal...",R,5.8,79.0,69.0,1.0,NaN,"$70,632",08 May 2020,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,The Con-Heartist,"Comedy, Romance","Romantic Comedies,Comedies,Romantic Films,Thai...",Thai,Movie,8.6,Thailand,> 2 hrs,Mez Tharatorn,"Pattaranad Bhiboonsawade, Mez Tharatorn, Thods...","Kathaleeya McIntosh, Nadech Kugimiya, Pimchano...",NaN,7.4,NaN,NaN,NaN,NaN,NaN,03 Dec 2020,2021-03-03,NaN,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,After her ex-boyfriend cons her out of a large...,131.0,https://occ-0-2188-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BODAzOG...,https://www.youtube.com/watch?v=md3CmFLGK6Y,YouTube
3,Gleboka woda,Drama,"TV Dramas,Polish TV Shows,Social Issue TV Dramas",Polish,Series,8.7,Poland,< 30 minutes,NaN,NaN,"Katarzyna Maciag, Piotr Nowak, Marcin Dorocins...",NaN,7.5,NaN,NaN,2.0,4.0,NaN,14 Jun 2011,2021-03-03,NaN,https://www.netflix.com/watch/81307527,https://www.imdb.com/title/tt2300049,A group of social welfare workers led by their...,47.0,https://occ-0-2508-2706.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BMTc0Nz...,https://www.youtube.com/watch?v=5kyF2vy63r0,YouTube
4,Only a Mother,Drama,"Social Issue Dramas,Dramas,Movies Based on Boo...",Swedish,Movie,8.3,"Lithuania,Poland,France,Italy,Spain,Greece,Bel...",1-2 hour,Alf Sjöberg,Ivar Lo-Johansson,"Hugo Björne, Eva Dahlbeck, Ulf Palme, Ragnar F...",NaN,6.7,NaN,NaN,2.0,1.0,NaN,31 Oct 1949,2021-03-03,NaN,https://www.netflix.com/watch/81382068,https://www.imdb.com/title/tt0041155,An unhappily married farm worker struggling to...,88.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BMjVmMz...,https://www.youtube.com/watch?v=H0itWKFwMpQ,YouTube


In [26]:
movie.shape

(9425, 29)

In [27]:
movie.columns

Index(['Title', 'Genre', 'Tags', 'Languages', 'Series or Movie',
       'Hidden Gem Score', 'Country Availability', 'Runtime', 'Director',
       'Writer', 'Actors', 'View Rating', 'IMDb Score',
       'Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received',
       'Awards Nominated For', 'Boxoffice', 'Release Date',
       'Netflix Release Date', 'Production House', 'Netflix Link', 'IMDb Link',
       'Summary', 'IMDb Votes', 'Image', 'Poster', 'TMDb Trailer',
       'Trailer Site'],
      dtype='object')

### Data Cleaning

In [28]:
## Changing variable types for all necessary columns

movie['Languages'] = movie['Languages'].astype(str)
movie['Genre'] = movie['Genre'].astype(str)
movie['Tags'] = movie['Tags'].astype(str)
movie['Actors'] = movie['Actors'].astype(str)
movie['Runtime'] = movie['Runtime'].astype(str)
movie['View Rating'] = movie['View Rating'].astype(str)
movie['Release Date'] = movie['Release Date'].astype(str)

In [29]:
## Changing the Languages, Genre, Tags, and Actors variables to lists

for i in range(0, movie.shape[0]):
    movie.at[i, 'Languages'] = movie.at[i, 'Languages'].split(", ")
    movie.at[i, 'Genre'] = movie.at[i, 'Genre'].split(", ")
    movie.at[i, 'Tags'] = movie.at[i, 'Tags'].split(",")
    movie.at[i, 'Actors'] = movie.at[i, 'Actors'].split(", ")

In [30]:
## Dropping unnecessary columns
movie = movie.drop(columns = ['Country Availability', 'Writer', 'Netflix Release Date', 'Production House', 'IMDb Votes', 
                             'Trailer Site', 'Boxoffice'])

In [31]:
## Creating a new data point for each "Language" option using the explode() function from Pandas

movie = movie.explode('Languages').reset_index(drop = True)

In [32]:
## Dropping observations missing a languages 

movie = movie[movie['Languages'] != 'nan']

## Resetting the index
movie = movie.reset_index(drop = True)

In [33]:
## Only keeping languages with more than 100 observations

top_languages = ['English', 'Japanese', 'Spanish', 'French', 'Korean', 'German', 'Hindi', 'Mandarin', 
                'Italian', 'Russian', 'Arabic', 'Cantonese', 'Portuguese', 'Thai', 'Dutch']


for i in range(0, movie.shape[0]):
    
    ## If the movie language is not a top language:
    if np.isin(movie['Languages'][i], top_languages, invert = True):
        
        ## Drop that observation from the data set
        movie = movie.drop([i])

## Resetting the index
movie = movie.reset_index(drop = True)

### Extra Cleaning

In [34]:
## Runtime
movie.at[8447, 'Runtime'] = '30-60 mins'

In [35]:
## View Rating
movie['View Rating'] = np.where(movie['View Rating'] == 'NOT RATED', 'Not Rated', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'Unrated', 'Not Rated', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'nan', 'Not Rated', movie['View Rating'])

movie['View Rating'] = np.where(movie['View Rating'] == 'X', 'NC-17', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'MA-17', 'NC-17', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'M/PG', 'R', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'Approved', 'PG-13', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'Passed', 'PG-13', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'GP', 'PG', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'E', 'G', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'AL', 'G', movie['View Rating'])

movie['View Rating'] = np.where(movie['View Rating'] == 'TV-Y', 'G', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-Y7', 'G', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-Y7-FV', 'G', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-G', 'G', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-PG', 'PG', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-13', 'PG-13', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-14', 'PG-13', movie['View Rating'])
movie['View Rating'] = np.where(movie['View Rating'] == 'TV-MA', 'NC-17', movie['View Rating'])

### Variable Engineering

In [36]:
movie.head()

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Runtime,Director,Actors,View Rating,IMDb Score,Rotten Tomatoes Score,Metacritic Score,Awards Received,Awards Nominated For,Release Date,Netflix Link,IMDb Link,Summary,Image,Poster,TMDb Trailer
0,Lets Fight Ghost,"[Crime, Drama, Fantasy, Horror, Romance]","[Comedy Programmes, Romantic TV Comedies, Horr...",Spanish,Series,4.3,< 30 minutes,Tomas Alfredson,"[Lina Leandersson, Kåre Hedebrant, Per Ragnar,...",R,7.9,98.0,82.0,74.0,57.0,12 Dec 2008,https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,https://www.youtube.com/watch?v=LqB6XJix-dM
1,HOW TO BUILD A GIRL,[Comedy],"[Dramas, Comedies, Films Based on Books, British]",English,Movie,7.0,1-2 hour,Coky Giedroyc,"[Cleo, Paddy Considine, Beanie Feldstein, Dóna...",R,5.8,79.0,69.0,1.0,NaN,08 May 2020,https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ
2,The Con-Heartist,"[Comedy, Romance]","[Romantic Comedies, Comedies, Romantic Films, ...",Thai,Movie,8.6,> 2 hrs,Mez Tharatorn,"[Kathaleeya McIntosh, Nadech Kugimiya, Pimchan...",Not Rated,7.4,NaN,NaN,NaN,NaN,03 Dec 2020,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,After her ex-boyfriend cons her out of a large...,https://occ-0-2188-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BODAzOG...,https://www.youtube.com/watch?v=md3CmFLGK6Y
3,Snowroller,[Comedy],"[Sports Movies, Sports Comedies, Comedies, Swe...",English,Movie,5.3,1-2 hour,Lasse Åberg,"[Lasse Åberg, Cecilia Walton, Eva Millberg, Jo...",Not Rated,6.6,NaN,NaN,NaN,NaN,04 Oct 1985,https://www.netflix.com/watch/81382187,https://www.imdb.com/title/tt0090115,"Two friends take a ski trip to the Alps, where...",https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BZDY2NG...,https://www.youtube.com/watch?v=tjWouBLwe3c
4,Snowroller,[Comedy],"[Sports Movies, Sports Comedies, Comedies, Swe...",German,Movie,5.3,1-2 hour,Lasse Åberg,"[Lasse Åberg, Cecilia Walton, Eva Millberg, Jo...",Not Rated,6.6,NaN,NaN,NaN,NaN,04 Oct 1985,https://www.netflix.com/watch/81382187,https://www.imdb.com/title/tt0090115,"Two friends take a ski trip to the Alps, where...",https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BZDY2NG...,https://www.youtube.com/watch?v=tjWouBLwe3c


In [37]:
## Average Score

## Putting all scores on the same scale
movie['Hidden Gem Score'] = movie['Hidden Gem Score'] * 10
movie['IMDb Score'] = movie['IMDb Score'] * 10

## Calculating the average of all scores
movie['Score'] = movie[['Hidden Gem Score', 'IMDb Score', 'Rotten Tomatoes Score', 'Metacritic Score']].mean(axis = 1)

In [38]:
movie['Score'].describe()

count    12109.000000
mean        62.935633
std         15.116867
min         10.250000
25%         54.000000
50%         65.500000
75%         75.000000
max         95.500000
Name: Score, dtype: float64

In [39]:
## Runtime Categoricals

movie['Runtime_1'] = np.where(movie['Runtime'] == '< 30 minutes', 1, 0)
movie['Runtime_2'] = np.where(movie['Runtime'] == '30-60 mins', 1, 0)
movie['Runtime_3'] = np.where(movie['Runtime'] == '1-2 hour', 1, 0)
movie['Runtime_4'] = np.where(movie['Runtime'] == '> 2 hrs', 1, 0)

In [40]:
movie['Runtime'].value_counts()

1-2 hour        6812
< 30 minutes    2812
> 2 hrs         2350
30-60 mins       135
Name: Runtime, dtype: int64

In [41]:
## View Rating Categoricals

movie['Rating_G'] = np.where(movie['View Rating'] == 'G', 1, 0)
movie['Rating_PG'] = np.where(movie['View Rating'] == 'PG', 1, 0)
movie['Rating_PG-13'] = np.where(movie['View Rating'] == 'PG-13', 1, 0)
movie['Rating_R'] = np.where(movie['View Rating'] == 'R', 1, 0)
movie['Rating_NC-17'] = np.where(movie['View Rating'] == 'NC-17', 1, 0)
movie['Rating_Not_Rated'] = np.where(movie['View Rating'] == 'Not Rated', 1, 0)

In [42]:
## Release Year

movie['Release'] = 0
movie['Release'] = movie['Release'].astype(str)

for i in range(0, movie.shape[0]):
    movie.at[i, 'Release'] = movie.at[i, 'Release Date'][-4:]

In [43]:
## Release Decade

movie['Release'] = np.where(movie['Release'] == 'nan', 0, movie['Release'])
movie['Release'] = movie['Release'].astype(int)

movie['Release_0'] = np.where((movie['Release'] >= 1900) & (movie['Release'] < 1940), 1, 0)
movie['Release_40'] = np.where((movie['Release'] >= 1940) & (movie['Release'] < 1950), 1, 0)
movie['Release_50'] = np.where((movie['Release'] >= 1950) & (movie['Release'] < 1960), 1, 0)
movie['Release_60'] = np.where((movie['Release'] >= 1960) & (movie['Release'] < 1970), 1, 0)
movie['Release_70'] = np.where((movie['Release'] >= 1970) & (movie['Release'] < 1980), 1, 0)
movie['Release_80'] = np.where((movie['Release'] >= 1980) & (movie['Release'] < 1990), 1, 0)
movie['Release_90'] = np.where((movie['Release'] >= 1990) & (movie['Release'] < 2000), 1, 0)
movie['Release_00'] = np.where((movie['Release'] >= 2000) & (movie['Release'] < 2010), 1, 0)
movie['Release_10'] = np.where((movie['Release'] >= 2010) & (movie['Release'] < 2020), 1, 0)
movie['Release_20'] = np.where((movie['Release'] >= 2020) & (movie['Release'] < 2030), 1, 0)

In [44]:
## Genre

## Initializing all genre values to zero
movie['Genre_Crime'] = 0
movie['Genre_Drama'] = 0
movie['Genre_Fantasy'] = 0
movie['Genre_Horror'] = 0
movie['Genre_Romance'] = 0
movie['Genre_Comedy'] = 0
movie['Genre_Mystery'] = 0
movie['Genre_Thriller'] = 0
movie['Genre_Short'] = 0
movie['Genre_Action'] = 0
movie['Genre_Adventure'] = 0
movie['Genre_Sci-Fi'] = 0
movie['Genre_Animation'] = 0
movie['Genre_Family'] = 0
movie['Genre_Biography'] = 0
movie['Genre_War'] = 0
movie['Genre_History'] = 0
movie['Genre_Documentary'] = 0
movie['Genre_Film-Noir'] = 0
movie['Genre_Sport'] = 0
movie['Genre_Game-Show'] = 0
movie['Genre_Western'] = 0
movie['Genre_Music'] = 0
movie['Genre_Musical'] = 0
movie['Genre_Reality-TV'] = 0
movie['Genre_News'] = 0
movie['Genre_Talk-Show'] = 0
movie['Genre_Adult'] = 0

## Setting genres to 1 if applicable
for i in range(0, movie.shape[0]):
    for j in range(0, len(movie['Genre'][i])):
        if movie['Genre'][i][j] == 'Crime':
            movie.at[i, 'Genre_Crime'] = 1
        elif movie['Genre'][i][j] == 'Drama':
            movie.at[i, 'Genre_Drama'] = 1
        elif movie['Genre'][i][j] == 'Fantasy':
            movie.at[i, 'Genre_Fantasy'] = 1
        elif movie['Genre'][i][j] == 'Horror':
            movie.at[i, 'Genre_Horror'] = 1
        elif movie['Genre'][i][j] == 'Romance':
            movie.at[i, 'Genre_Romance'] = 1
        elif movie['Genre'][i][j] == 'Comedy':
            movie.at[i, 'Genre_Comedy'] = 1
        elif movie['Genre'][i][j] == 'Mystery':
            movie.at[i, 'Genre_Mystery'] = 1
        elif movie['Genre'][i][j] == 'Thriller':
            movie.at[i, 'Genre_Thriller'] = 1
        elif movie['Genre'][i][j] == 'Short':
            movie.at[i, 'Genre_Short'] = 1
        elif movie['Genre'][i][j] == 'Action':
            movie.at[i, 'Genre_Action'] = 1
        elif movie['Genre'][i][j] == 'Adventure':
            movie.at[i, 'Genre_Adventure'] = 1
        elif movie['Genre'][i][j] == 'Sci-Fi':
            movie.at[i, 'Genre_Sci-Fi'] = 1
        elif movie['Genre'][i][j] == 'Animation':
            movie.at[i, 'Genre_Animation'] = 1
        elif movie['Genre'][i][j] == 'Family':
            movie.at[i, 'Genre_Family'] = 1
        elif movie['Genre'][i][j] == 'Biography':
            movie.at[i, 'Genre_Biography'] = 1
        elif movie['Genre'][i][j] == 'War':
            movie.at[i, 'Genre_War'] = 1
        elif movie['Genre'][i][j] == 'History':
            movie.at[i, 'Genre_History'] = 1
        elif movie['Genre'][i][j] == 'Documentary':
            movie.at[i, 'Genre_Documentary'] = 1
        elif movie['Genre'][i][j] == 'Film-Noir':
            movie.at[i, 'Genre_Film-Noir'] = 1
        elif movie['Genre'][i][j] == 'Sport':
            movie.at[i, 'Genre_Sport'] = 1
        elif movie['Genre'][i][j] == 'Game-Show':
            movie.at[i, 'Genre_Game-Show'] = 1
        elif movie['Genre'][i][j] == 'Western':
            movie.at[i, 'Genre_Western'] = 1
        elif movie['Genre'][i][j] == 'Music':
            movie.at[i, 'Genre_Music'] = 1
        elif movie['Genre'][i][j] == 'Musical':
            movie.at[i, 'Genre_Musical'] = 1
        elif movie['Genre'][i][j] == 'Reality-TV':
            movie.at[i, 'Genre_Reality-TV'] = 1
        elif movie['Genre'][i][j] == 'News':
            movie.at[i, 'Genre_News'] = 1
        elif movie['Genre'][i][j] == 'Talk-Show':
            movie.at[i, 'Genre_Talk-Show'] = 1
        elif movie['Genre'][i][j] == 'Adult':
            movie.at[i, 'Genre_Adult'] = 1

In [45]:
## Directors (Top-10)


In [46]:
movie['Director'].value_counts().head(10)

Steven Spielberg     65
Ridley Scott         41
Johnnie To           34
Steven Soderbergh    34
Jay Roach            30
Tsutomu Shibayama    29
Woody Allen          28
Roland Emmerich      27
Ron Howard           26
Quentin Tarantino    25
Name: Director, dtype: int64

In [47]:
## Dropping unnecessary columns after engineering process
movie = movie.drop(columns = ['Release Date', 'Release', 'Hidden Gem Score', 'Runtime', 'IMDb Score', 'Rotten Tomatoes Score', 
                             'Metacritic Score', 'Tags', 'Awards Received', 'Awards Nominated For'])

In [24]:
movie.to_csv('blockbuster.csv', index = False)

In [62]:
blockbuster = movie

In [63]:
 ## Removing unnecessary variables for recommendation process
blockbuster_temp = blockbuster.iloc[0:100, 14:62]
blockbuster_temp.head()

,Runtime_1,Runtime_2,Runtime_3,Runtime_4,Rating_G,Rating_PG,Rating_PG-13,Rating_R,Rating_NC-17,Rating_Not_Rated,Release_0,Release_40,Release_50,Release_60,Release_70,Release_80,Release_90,Release_00,Release_10,Release_20,Genre_Crime,Genre_Drama,Genre_Fantasy,Genre_Horror,Genre_Romance,Genre_Comedy,Genre_Mystery,Genre_Thriller,Genre_Short,Genre_Action,Genre_Adventure,Genre_Sci-Fi,Genre_Animation,Genre_Family,Genre_Biography,Genre_War,Genre_History,Genre_Documentary,Genre_Film-Noir,Genre_Sport,Genre_Game-Show,Genre_Western,Genre_Music,Genre_Musical,Genre_Reality-TV,Genre_News,Genre_Talk-Show,Genre_Adult
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
blockbuster_temp.shape

(100, 48)

In [66]:
from sklearn.metrics.pairwise import euclidean_distances

## Computing the Euclidean distances for all observations
D = euclidean_distances(blockbuster_temp)

In [97]:
D = euclidean_distances(blockbuster_temp)

results = pd.DataFrame(columns = ['Rec_1', 'Rec_2', 'Rec_3', 'Rec_4', 'Rec_5'])

for i in range(0, blockbuster_temp.shape[0]):
    
    top_5 = np.argsort(D[:, i])[1:6]
    results.loc[i] = [blockbuster.loc[top_5[0], 'Title'], blockbuster.loc[top_5[1], 'Title'], blockbuster.loc[top_5[2], 'Title'], 
                      blockbuster.loc[top_5[3], 'Title'], blockbuster.loc[top_5[4], 'Title']] 

In [98]:
results

,Rec_1,Rec_2,Rec_3,Rec_4,Rec_5
0,Intimate Strangers,The Coroner,Sway,The New Pope,Burden
1,I Care a Lot,Burden,The Closet,The Loop,Baumbacher Syndrome
2,La Palma,La Palma,La Palma,Floating Clouds,Baumbacher Syndrome
3,Snowroller,The Loop,5 Minute,Baumbacher Syndrome,Happy Bus Day
4,Snowroller,The Loop,5 Minute,Baumbacher Syndrome,Happy Bus Day
5,Angels Time,Sway,The Big Shot,Harrys Daughters,The Return
6,Oh My JUMP!,1987: When the Day Comes,Ten Years Japan,Default,Default
7,The Girl on the Train,The Girl on the Train,Default,Default,I Care a Lot
8,Angels Time,Wonderstruck,Wonderstruck,Dilili in Paris,Old Marine Boy
9,Wonderstruck,Wonderstruck,Miss Baek,The Bold Type,The Invisible


In [86]:
D = euclidean_distances(blockbuster_temp)

In [87]:
D

array([[0.        , 3.16227766, 3.31662479, ..., 3.        , 3.        ,
        3.31662479],
       [3.16227766, 0.        , 2.23606798, ..., 2.64575131, 2.64575131,
        3.        ],
       [3.31662479, 2.23606798, 0.        , ..., 2.44948974, 2.44948974,
        2.82842712],
       ...,
       [3.        , 2.64575131, 2.44948974, ..., 0.        , 0.        ,
        2.44948974],
       [3.        , 2.64575131, 2.44948974, ..., 0.        , 0.        ,
        2.44948974],
       [3.31662479, 3.        , 2.82842712, ..., 2.44948974, 2.44948974,
        0.        ]])

In [92]:
np.argsort(D[:, 0])[1:6]

array([55, 11, 20, 31, 91])

In [67]:
np.argsort(D)

array([[ 0, 55, 11, ..., 18, 29, 17],
       [ 1, 90, 91, ..., 46, 44, 80],
       [ 2, 75, 76, ..., 64, 80,  5],
       ...,
       [97, 98, 57, ..., 17, 81, 82],
       [97, 98, 57, ..., 17, 81, 82],
       [99, 34, 26, ..., 80, 84, 83]])

In [79]:
#pd.DataFrame(D)

In [69]:
to_select = np.argsort(D[:, 0])[1:6]

In [70]:
to_select

array([55, 11, 20, 31, 91])

In [72]:
[blockbuster.loc[to_select[0], 'Title'], blockbuster.loc[to_select[1], 'Title'], 
 blockbuster.loc[to_select[2], 'Title'], blockbuster.loc[to_select[3], 'Title'], 
 blockbuster.loc[to_select[4], 'Title']] 

['Intimate Strangers', 'The Coroner', 'Sway', 'The New Pope', 'Burden']

In [71]:
blockbuster.head()

,Title,Genre,Languages,Series or Movie,Director,Actors,View Rating,Netflix Link,IMDb Link,Summary,Image,Poster,TMDb Trailer,Score,Runtime_1,Runtime_2,Runtime_3,Runtime_4,Rating_G,Rating_PG,Rating_PG-13,Rating_R,Rating_NC-17,Rating_Not_Rated,Release_0,Release_40,Release_50,Release_60,Release_70,Release_80,Release_90,Release_00,Release_10,Release_20,Genre_Crime,Genre_Drama,Genre_Fantasy,Genre_Horror,Genre_Romance,Genre_Comedy,Genre_Mystery,Genre_Thriller,Genre_Short,Genre_Action,Genre_Adventure,Genre_Sci-Fi,Genre_Animation,Genre_Family,Genre_Biography,Genre_War,Genre_History,Genre_Documentary,Genre_Film-Noir,Genre_Sport,Genre_Game-Show,Genre_Western,Genre_Music,Genre_Musical,Genre_Reality-TV,Genre_News,Genre_Talk-Show,Genre_Adult
0,Lets Fight Ghost,"[Crime, Drama, Fantasy, Horror, Romance]",Spanish,Series,Tomas Alfredson,"[Lina Leandersson, Kåre Hedebrant, Per Ragnar,...",R,https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,https://www.youtube.com/watch?v=LqB6XJix-dM,75.5,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOW TO BUILD A GIRL,[Comedy],English,Movie,Coky Giedroyc,"[Cleo, Paddy Considine, Beanie Feldstein, Dóna...",R,https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,69.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Con-Heartist,"[Comedy, Romance]",Thai,Movie,Mez Tharatorn,"[Kathaleeya McIntosh, Nadech Kugimiya, Pimchan...",Not Rated,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,After her ex-boyfriend cons her out of a large...,https://occ-0-2188-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BODAzOG...,https://www.youtube.com/watch?v=md3CmFLGK6Y,80.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Snowroller,[Comedy],English,Movie,Lasse Åberg,"[Lasse Åberg, Cecilia Walton, Eva Millberg, Jo...",Not Rated,https://www.netflix.com/watch/81382187,https://www.imdb.com/title/tt0090115,"Two friends take a ski trip to the Alps, where...",https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BZDY2NG...,https://www.youtube.com/watch?v=tjWouBLwe3c,59.5,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Snowroller,[Comedy],German,Movie,Lasse Åberg,"[Lasse Åberg, Cecilia Walton, Eva Millberg, Jo...",Not Rated,https://www.netflix.com/watch/81382187,https://www.imdb.com/title/tt0090115,"Two friends take a ski trip to the Alps, where...",https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BZDY2NG...,https://www.youtube.com/watch?v=tjWouBLwe3c,59.5,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
